# Sudoku solver with statistics

Author: Frankie Inguanez <br />
Date: 15/01/2023<br /><br />

An backtracking algorithm for a 9x9 sudoku puzzle taking.

In [1]:
import sudokuSearchAlg as ssa
import sudokuGuessAlg as sga
import sudokuPuzzleUtils as spu

In [2]:
class SudokuStats:
    def __init__(self):
        self.guesses = 0
        self.backtracks = 0
        self.startTime = None
        self.endTime = None

    def incrementGuesses(self):
        self.guesses += 1

    def incrementBacktracks(self):
        self.backtracks += 1

    def registerStartTime(self):
        import time

        self.startTime = time.time()

    def registerEndTime(self):
        import time

        self.endTime = time.time()

    def guesses(self):
        return self.guesses

    def backtracks(self):
        return self.backtracks

    def executionTime(self):
        return self.endTime-self.startTime

In [3]:
def backtracking(board, stats: SudokuStats, searchMode: int, guessMode: int):
    """
    Solves a 9x9 sudoku puzzle using backtracking algorithm.
    Arguments:
        board: the 9x9 puzzle to be solved.
        stats: The statistics object to record algorithm.
        searchMode: defines how the puzzle is parsed: 1 by row; 2 by col; 3 by box sequentially; 4 by box in a zig-zag; 5 by box in a spiral; 6 by box in a semi-zig-zag
        guessMode: defines how numbers are guessed: 1 sequentially; 2 randomly
    """
    # Find the next empty cell
    find = ssa.findEmpty(board, searchMode)

    # If there is no empty cell than puzzle is complete
    if not find:
        stats.registerEndTime()
        return True
    else:
        row, col = find

    # Get numbers to guess and attempt
    for guess in sga.getGuesses(board, guessMode):
        if spu.isValid(board, guess, (row, col)):

            # Brute force guess
            stats.incrementGuesses()
            board[row][col] = guess

            # Attempt to solve rest of puzzle with current choice
            if backtracking(board, stats, searchMode, guessMode):
                return True

            # Invalid puzzle so backtrack
            stats.incrementBacktracks()
            board[row][col] = 0

    return False

In [4]:
board = spu.to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')
board

[[1, 0, 0, 0, 0, 8, 5, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 6, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 5, 0, 2, 0, 0, 4, 7],
 [0, 9, 4, 6, 0, 0, 3, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 9, 0],
 [9, 2, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 5]]

In [5]:
ssa.findRandom(board)

(4, 5)

In [6]:
# Register the start time
stats = SudokuStats()
stats.registerStartTime()
print("Solved puzzle!" if backtracking(board, stats, 6, 1) else "Could not solve puzzle")
stats.registerEndTime()

board

Solved puzzle!


[[1, 3, 7, 4, 6, 8, 5, 2, 9],
 [2, 5, 8, 3, 1, 9, 4, 7, 6],
 [4, 6, 9, 5, 7, 2, 1, 8, 3],
 [6, 1, 2, 7, 3, 4, 9, 5, 8],
 [3, 8, 5, 9, 2, 1, 6, 4, 7],
 [7, 9, 4, 6, 8, 5, 3, 1, 2],
 [5, 7, 6, 2, 4, 3, 8, 9, 1],
 [9, 2, 1, 8, 5, 6, 7, 3, 4],
 [8, 4, 3, 1, 9, 7, 2, 6, 5]]

In [7]:
stats.executionTime()

0.7237107753753662

In [8]:
stats.backtracks

61038

In [9]:
stats.guesses

61094

In [10]:
def solvePuzzles(puzzlesFileName, statsFileName, errorsFileName, limit, searchMode, guessMode):
    """
    Solves puzzles found in a file using backtracking algorithm.
    Arguments:
        puzzlesFileName: file containing puzzles in following format: <id>, <puzzle>, <solution>
        statsFileName: file where statistics shall be saved.
        errorsFileName: file where errors shall be saved.
        limit: the limit number of puzzles to solve.
        searchMode: defines how missing values are searched.
        guessMode: defines how guesses are made.
    """
    import tqdm

    if not limit:
        limit = spu.getFileLineCount(puzzlesFileName)

    i = 1
    hasError = False
    try:
        print("Starting sudoku base solver.")
        
        with open(statsFileName, "w", encoding="utf-8") as sf:
            with open(puzzlesFileName, "r", encoding="utf-8") as pf:
                for line in tqdm.tqdm(pf, total=limit):
                    # Stop at limit
                    if (i>limit):
                        break
                    
                    # Parse the content
                    data = line.split(',')
                    p = data[1].strip()

                    # Create board and statistics
                    board = spu.to2DArray(p)
                    stats = SudokuStats();

                    # Start statistics gathering and solve
                    stats.registerStartTime()
                    backtracking(board, stats, searchMode, guessMode)
                    stats.registerEndTime()
                        
                    # Write statistics
                    sf.write("{},{},{:0.22f},{:0.0f},{:0.0f}\n".format(p, spu.toStr(board), stats.executionTime(), stats.guesses, stats.backtracks))
                    i+=1

    except Exception as e:
        hasError = True
        spu.saveError(e, errorsFileName)

    print("Operation encountered some errors. Check {} for details or script output above.".format(errorsFileName) \
        if hasError else "Sudoku puzzles solved completed successfully.")
    print("Sudoku solver statistics saved in {}".format(statsFileName))

In [11]:
solvePuzzles("sudokuDataset.csv","backtracking.csv", "backtrackingErrors.txt", 10, 7, 1)

Starting sudoku base solver.


100%|██████████| 10/10 [00:01<00:00,  7.57it/s]

Sudoku puzzles solved completed successfully.
Sudoku solver statistics saved in backtracking.csv
